In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/foochuanyue/Desktop/aiap-14/all-assignments/assignment7/aiap-14-ds-llm-topic-01-sa.json'

In [2]:
%load_ext autoreload
%autoreload 2

import argparse
import os
from typing import List
from csv import writer
import sys

from dotenv import load_dotenv
from langchain.document_loaders import CSVLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from vertexai.preview.language_models import TextGenerationModel

In [3]:
sys.path.append("..")
from src.generative_agents.generative_agent import StemssGenerativeAgent
from src.generative_agents.memory import StemssGenerativeAgentMemory
from src.retrievers.time_weighted_retriever import ModTimeWeightedVectorStoreRetriever
from src.vectorstores.chroma import EnhancedChroma
from src.generators.schedule import generate_schedule
from src.generators.agent import generate_agent_name, generate_characters
from src.tools.action import force_dialogue

In [4]:
def load_documents() -> List[Document]:
    """ "Load memory (history) from CSV "

    Returns
    -------
    list
        A list of documents where each row of the CSV is a document
    """
    loader = CSVLoader(
        mem_file, metadata_columns=["last_accessed_at", "created_at", "importance"]
    )
    docs = loader.load()
    return docs


def create_new_memory_retriever(decay_rate: float = 0.5, k: int = 5, mem_file:str="./memory/memory.csv"):
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = VertexAIEmbeddings()
    vs = EnhancedChroma(embedding_function=embeddings_model)
    return ModTimeWeightedVectorStoreRetriever(
        vectorstore=vs, other_score_keys=["importance"], decay_rate=decay_rate, k=k, mem_file=mem_file
    )


In [5]:
# Load the .env file
load_dotenv()

# Get the path of Google Cloud credentials from the environment variable
google_creds_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

In [6]:
path='../outputs'
all_folders = os.listdir(path)
new = 1
if len(all_folders)!=0:
    all_folders.sort()
    latest = all_folders[-1].replace('run_', '')
    new = int(latest) + 1
new_path = f"{path}/run_{new}"
os.makedirs(new_path)
os.makedirs(f"{new_path}/memory")


agents = []
#Generate agent
llm = VertexAI(model_name="text-bison@001", max_output_tokens=256, temperature=0.2)
agent_names = generate_agent_name(model=llm, num_of_agents=2)
for agent_name in agent_names:

    llm = VertexAI(model_name="text-bison@001", max_output_tokens=256, temperature=0.7)
    #generate agent details
    agent_details = generate_characters(model=llm, agent_name=agent_name)
    agent_details['name']=agent_name

    #Create csv if it doesn't exist
    mem_file = f"{new_path}/memory/{agent_name}.csv"
    if  not os.path.exists(mem_file):
        with open(mem_file, 'a') as f_object:
            List = ["created_at", "last_accessed_at", "observations", "importance"]
            writer_object = writer(f_object)
            writer_object.writerow(List)
            f_object.close()

    #Load CSV
    docs = load_documents()
    memory_retriever = create_new_memory_retriever(decay_rate=0.5, k=5 ,mem_file=mem_file)
    if(len(docs)!=0):
        memory_retriever.add_documents(docs)

    agent_memory = StemssGenerativeAgentMemory(
        llm=llm,
        memory_retriever=memory_retriever,
        reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
        verbose=True,
    )

    agent = StemssGenerativeAgent(
        name=agent_name,
        age=agent_details["age"],
        traits=agent_details["traits"],
        status="",
        memory_retriever=create_new_memory_retriever(),
        llm=llm,
        memory=agent_memory,
        background=agent_details["background"],
    )
    agent.get_summary()
    agents.append(agent)

In [9]:
observations = generate_schedule(model=llm, agent=agents[0])
for observation in observations:
    agents[0].add_memory(observation)



> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Alex observed Plan to finish the pending conversation with your spouse at home
Rating: 


Importance score: 8



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Alex observed Plan to have a talk with your spouse about the upcoming meeting
Rating: 


Importance score: 8



> Finished chain.


In [7]:
agents[0].get_summary()

'Name: Alex (age: 25)\nInnate traits: ambitious, charming, impatient\nAlex is passionate and driven to help others, but she can sometimes be impatient.'

In [8]:
agents[1].get_summary()

'Name: Ben (age: 32)\nInnate traits: curious, shy, optimistic\nBen is a shy and introverted librarian who loves to read and help others.'

In [11]:
agents[0].generate_dialogue_response("Hello")

Name: Alex (age: 25)
Innate traits: ambitious, charming, impatient
Alex is passionate and driven to help others, but she can sometimes be impatient.


Number of requested results 100 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 100 is greater than number of elements in index 2, updating n_results = 2


Name: Alex (age: 25)
Innate traits: ambitious, charming, impatient
Alex is passionate and driven to help others, but she can sometimes be impatient.
It is October 14, 2023, 06:06 PM.
Alex's status: 
Summary of relevant context from Alex's memory:
The correct answer is: 'creator'.

In the context, Alex is the creator of the two plans.
Most recent observations: []
Observation: Hello

What would Alex say? If the conversation is getting similar end it. To end the conversation, write: GOODBYE: "what to say". Otherwise to continue the conversation, write: SAY: "what to say next"




> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Alex observed Hello and said Hi. What can I help you with?
Rating: 


Importance score: 1



> Finished chain.


(True, 'Alex said Hi. What can I help you with?')

In [10]:
agents[1].generate_dialogue_response("Hello")

Name: Ben (age: 32)
Innate traits: curious, shy, optimistic
Ben is a shy and introverted librarian who loves to read and help others.


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Name: Ben (age: 32)
Innate traits: curious, shy, optimistic
Ben is a shy and introverted librarian who loves to read and help others.
It is October 14, 2023, 06:05 PM.
Ben's status: 
Summary of relevant context from Ben's memory:
Ben is the name of a person. Ben is a person who greets others. Hello is an interjection used as a greeting or salutation. Ben uses hello to greet others. So the relationship between Ben and hello is that hello is used by Ben to greet others.
Most recent observations: []
Observation: Hello

What would Ben say? If the conversation is getting similar end it. To end the conversation, write: GOODBYE: "what to say". Otherwise to continue the conversation, write: SAY: "what to say next"




> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You mu

Importance score: 1



> Finished chain.


(True, 'Ben said Hello to you too. What can I help you with?')

In [17]:
agents[1].add_memory('test')



> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Bob observed test
Rating: 


Importance score: 5



> Finished chain.


In [10]:
force_dialogue(agents[0], agents[1])

Name: Alex (age: 23)
Innate traits: cynical, sarcastic, smart
Alex is a recent college graduate working as a software engineer at a local startup. He is a bit of a loner and doesn't really have any close friends in AI Town yet. He is very focused on his work and enjoys starting his day early with a cup of coffee and checking his email.


KeyboardInterrupt: 